# Demo access to DISPATCH format ``stars.dat``  files
Do NOT run and update this notebook itself (except when actually amending or improving it).   Instead, use the "Duplicate" option to make a copy, and use that to do visual validation.

In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Standard Python startup

In [3]:
top='/groups/astro/aake/codes/dispatch2/'            # aake
#top='/groups/astro/xxxx/codes/dispatch2/'            # xxxx

In [42]:
import sys
sys.path.insert(0,top+'utilities/python')
import dispatch
from scipy.io import FortranFile as ff

In [43]:
import os
dir=top+'experiments/ISM/python'
os.chdir(dir)
print('working directory:',dir)

working directory: /groups/astro/aake/codes/dispatch2/experiments/ISM/python


## Definition of Sinks() object

In [16]:
class Record(object):
  """ Template object for a file record """
  pass

class Sinks(object):
  """ Object with methods that can read DISPATCH and RAMSES stars.dat files,
      returning a dictionary where the values are lists of sink records,
      and the keys are the sink IDs
  """
  def __init__(self,io=0,run='',data='../data/',file='stars.dat',iomax=1000,verbose=0):
    """ Initial setup
    """
    self.sinks={}
    if io=='all':
      ios=[]
      for io in range(iomax):
        filename=data+run+'/{:05d}/'.format(io)+file
        if os.path.isfile(filename):
          ios.append(io)
      for io in ios:
        self.read(io,run,data,file,verbose)
    else:
      self.read(io,run,data,file,verbose)
    # Cache the available sink IDs
    self.ids=list(self.sinks.keys())
  
  def read(self,io,run,data,file,verbose):
    """ Read sinks for the interval starting with snaposhot io
    """
    # Construct the filename and read the file format
    self.filename=data+run+'/{:05d}/'.format(io)+file
    with ff(self.filename) as fd:
      fmt=fd.read_record('i4')[0]
    self.format='RAMSES' if fmt>0 else 'DISPATCH'
    if verbose>0:
      print('reading',self.filename)
      print('file format:',self.format)
    
    # Get a Fortran file object and read the file until the end    
    with ff(self.filename) as fd:
      try:
        while (True):
          if fmt>0:
            self.read_ramses(fd,verbose=verbose)
          else:
            self.read_dispatch(fd,verbose=verbose)
      except:
        pass
  
  def read_dispatch(self,fd,verbose=0):
    """ Read one snaphot of DISPATCH format sink data
    """
    # Try reading DISPATCH format sinks
    h=Record()
    h.io_format,h.nstars,h.time,h.nmetal=fd.read_record('i4,i4,f8,i4')[0]
    if verbose>1:
      print('format:',h.io_format,' nstars:',h.nstars)

    # If reading the header record worked, read the sink records
    for i in range(h.nstars):
      r=Record()
      r.header=h
      id,r.patch_id,r.level, \
      r.position,r.velocity,r.mass,r.rho,r.phi,r.dmdt,r.t_create,r.t_explode=\
        fd.read_record('i4,i4,i4,(3)f8,(3)f8,f8,f4,f4,f4,f8,f8')[0]
      r.id=id-1
      if verbose>1:
        print('id:',r.id,' patch:',r.patch_id,' mass:',r.mass,' position:',r.position)
      if not (r.id in self.sinks.keys()):
        self.sinks[r.id]=[]
      sink=self.sinks[r.id]
      sink.append(r)
  
  def read_ramses(self,fd,verbose=0):
    """ Read one snaphot of RAMSES format sink data
    """
    # Read header records
    h=Record()
    h.io_format,h.nstars,h.time4,h.szstar,h.sizeof\
      =fd.read_record('i4,i4,f4,i4,i4')[0]
    h.dp,h.nmetal,h.tflush,h.time=\
      fd.read_record('i4,i4,f8,f8')[0]
    h.do_translate,h.center_star,h.r_translate,h.v_translate,h.x_refine,h.y_refine,h.z_refine\
        =fd.read_record('i4,i4,(3)f8,(3)f8,f8,f8,f8')[0]
    h.do_translate=not h.do_translate==0
    # FIXME
    h.nstars=5
    # Read sinks
    for i in range(h.nstars):
      r=Record()
      r.id=i
      r.header=h
      r.x,r.y,r.z,r.px,r.py,r.pz,r.dpx,r.dpy,r.dpz,r.mass,\
        r.dmass,r.potential,r.density,r.t_create,r.t_explode,\
        r.gid,r.father,r.ilevel,r.old_ilevel,\
        r.owner,r.has_moved,r.beta_create,r.dtold\
        =fd.read_record('f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,\
                        f8,f8,f8,f8,f8,\
                        i4,i4,i4,i4,\
                        i4,i4,f8,f8')[0]
      if not (r.id in self.sinks.keys()):
        self.sinks[r.id]=[]
      sink=self.sinks[r.id]
      sink.append(r)

  def info(self,id=None):
    print('file:',self.filename)
    print('the file contains data for sinks',self.ids)
    if id is None:
      id=self.ids[0]
    snaps=self.sinks[id]
    print("the number of sink {} snapshots is {}".format(id,len(snaps)))
    sink=snaps[0]
    print('sink attributes:')
    for k,v in vars(sink).items():        # loop over attribues
      print('{:>12s} : {}'.format(k,v))   # print name and value
    print('sink.header attributes:')
    header=sink.header
    for k,v in vars(header).items():      # loop over header attribues
      print('{:>12s} : {}'.format(k,v))   # print name and value


In [12]:
S=Sinks(io=12,run='pp-test1-lv6',data='../data/')
S.info(1)

file: ../data/pp-test1-lv6/00012/stars.dat
the file contains data for sinks [0, 1, 2, 3, 4]
the number of sink 1 snapshots is 7
sink attributes:
      header : <__main__.Record object at 0x2b377bd1d700>
    patch_id : 123
       level : 6
    position : [-0.10073959  0.29596209  0.30164576]
    velocity : [-5.40825568 -4.67901768  0.30905386]
        mass : 0.011840166562374289
         rho : 0.0
         phi : 0.0
        dmdt : 0.0
    t_create : 0.0
   t_explode : 0.0
          id : 1
sink.header attributes:
   io_format : -1
      nstars : 5
        time : 0.12067320055378168
      nmetal : 0


In [17]:
S=dispatch.sinks(io=12,run='pp-test1-lv6',data='../data/')
S.info(1)

file: ../data/pp-test1-lv6/00012/stars.dat
the file contains data for sinks [0, 1, 2, 3, 4]
the number of sink 1 snapshots is 7
sink attributes:
      header : <dispatch._sinks.Record object at 0x2b377c7c3250>
sink.header attributes:
    patch_id : 123
sink.header attributes:
       level : 6
sink.header attributes:
    position : [-0.10073959  0.29596209  0.30164576]
sink.header attributes:
    velocity : [-5.40825568 -4.67901768  0.30905386]
sink.header attributes:
        mass : 0.011840166562374289
sink.header attributes:
         rho : 0.0
sink.header attributes:
         phi : 0.0
sink.header attributes:
        dmdt : 0.0
sink.header attributes:
    t_create : 0.0
sink.header attributes:
   t_explode : 0.0
sink.header attributes:
          id : 1
sink.header attributes:
   io_format : -1
      nstars : 5
        time : 0.12067320055378168
      nmetal : 0


In [76]:
S=dispatch.sinks(io='all',run='pp-test1-lv6',data='../data/')
S.info()

file: ../data/pp-test1-lv6/00049/stars.dat
the file contains data for sinks [0, 1, 2, 3, 4, 5]
the number of sink 0 snapshots is 245
sink attributes:
      header : <__main__.Record object at 0x2af722af6d00>
    patch_id : 25
       level : 6
    position : [-0.2578125  0.3046875 -0.3515625]
    velocity : [0. 0. 0.]
        mass : 0.00011894046269844694
         rho : 0.0
         phi : 0.0
        dmdt : 0.0
    t_create : 0.0
   t_explode : 0.0
          id : 0
sink.header attributes:
   io_format : -1
      nstars : 1
        time : 0.04884500585400099
      nmetal : 0


In [77]:
S=dispatch.sinks(io=12,run='vscode',data='../data/')
S.info()

file: ../data/vscode/00012/stars.dat
the file contains data for sinks [0, 1, 2, 3, 4]
the number of sink 0 snapshots is 6
sink attributes:
          id : 0
      header : <__main__.Record object at 0x2af721c92a60>
           x : -0.06446620798492012
           y : 0.2792084597693634
           z : -0.2649326829560591
          px : 0.13602237680431783
          py : -0.0786710206502615
          pz : 0.11489372643582763
         dpx : 0.0
         dpy : 0.0
         dpz : 0.0
        mass : 0.0162134363700089
       dmass : 0.0
   potential : 0.0
     density : 0.0
    t_create : 0.0
   t_explode : 0.0
         gid : 0
      father : 0
      ilevel : 0
  old_ilevel : 0
       owner : 0
   has_moved : 0
 beta_create : 0.0
       dtold : 0.0
sink.header attributes:
   io_format : 9
      nstars : 5
       time4 : 0.12057817727327347
      szstar : 0
      sizeof : 0
          dp : 8
      nmetal : 0
      tflush : 0.12
        time : 0.12057817842856695
do_translate : False
 center_star 

In [70]:
S=dispatch.sinks(io='all',run='vscode',data='../data/')
S.info()

file: ../data/vscode/00058/stars.dat
the file contains data for sinks [0, 1, 2, 3, 4]
the number of sink 0 snapshots is 245
sink attributes:
          id : 0
      header : <__main__.Record object at 0x2af721b264c0>
           x : -0.2578125
           y : 0.3046875
           z : -0.3515625
          px : 0.0
          py : 0.0
          pz : 0.0
         dpx : 0.0
         dpy : 0.0
         dpz : 0.0
        mass : 0.00011842990501342854
       dmass : 0.0
   potential : 0.0
     density : 0.0
    t_create : 0.0
   t_explode : 0.0
         gid : 0
      father : 0
      ilevel : 0
  old_ilevel : 0
       owner : 0
   has_moved : 0
 beta_create : 0.0
       dtold : 0.0
sink.header attributes:
   io_format : 9
      nstars : 5
       time4 : 0.04886363074183464
      szstar : 0
      sizeof : 0
          dp : 8
      nmetal : 0
      tflush : 0.0
        time : 0.0488636295220076
do_translate : False
 center_star : 0
 r_translate : [0. 0. 0.]
 v_translate : [0. 0. 0.]
    x_refine : 0

In [44]:
sn=dispatch.snapshot(12,'vscode')
sn.sinks.info()

file: ../data/vscode/00012/stars.dat
the file contains data for sinks [0, 1, 2, 3, 4]
the number of sink 0 snapshots is 6
sink attributes:
          id : 0
sink.header attributes:
        time : 0.12057817842856695
sink.header attributes:
      header : <dispatch._sinks.Record object at 0x2b377ccf4310>
sink.header attributes:
           x : -0.06446620798492012
sink.header attributes:
           y : 0.2792084597693634
sink.header attributes:
           z : -0.2649326829560591
sink.header attributes:
          px : 0.13602237680431783
sink.header attributes:
          py : -0.0786710206502615
sink.header attributes:
          pz : 0.11489372643582763
sink.header attributes:
         dpx : 0.0
sink.header attributes:
         dpy : 0.0
sink.header attributes:
         dpz : 0.0
sink.header attributes:
        mass : 0.0162134363700089
sink.header attributes:
       dmass : 0.0
sink.header attributes:
   potential : 0.0
sink.header attributes:
     density : 0.0
sink.header attributes:
  

In [45]:
sink_snapshots=sn.sinks.snapshots[1]
sink_snapshots[0].position
vars(sink_snapshots[0])
for s in sink_snapshots:
  print(s.time,s.position)

0.12057817842856695 [-0.07730751  0.3136274   0.29809725]
0.12057817842856695 [-0.07730751  0.3136274   0.29809725]
0.1228312972273877 [-0.08767321  0.3049681   0.29829055]
0.12513707890057912 [-0.098073    0.29575443  0.29814295]
0.1264296947393415 [-0.10846607  0.28597196  0.29761298]
0.12846795691669458 [-0.11880188  0.27561466  0.29664139]
